## Load necessary modules

In [2]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import glob

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load RetinaNet model

In [12]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('..', 'snapshots', 'sku_model_csv.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.load_model(model_path, backbone_name='resnet50', convert_model=True)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'sku'}

## Define Non-Maximum Suppression(NMS)

In [4]:
from numpy import *

def nms (boxes, score, overlap):
  if not len(boxes):
        pick = []
  else:
        trial = zeros((len(boxes),4),dtype=float64)
        trial[:] = boxes[:]
        x1 = trial[:,0]
        y1 = trial[:,1]
        x2 = trial[:,2]
        y2 = trial[:,3]
        area = (x2-x1+1)*(y2-y1+1)
        I = argsort(score)
        pick = []
        count = 1
        while (I.size!=0):
            last = I.size
            i = I[last-1]
            pick.append(i)
            suppress = [last-1]
            for pos in range(last-1):
                j = I[pos]
                xx1 = max(x1[i],x1[j])
                yy1 = max(y1[i],y1[j])
                xx2 = min(x2[i],x2[j])
                yy2 = min(y2[i],y2[j])
                w = xx2-xx1+1
                h = yy2-yy1+1
                if (w>0 and h>0):
                    o = w*h/area[j]
                    if (o >overlap):
                        suppress.append(pos)
            I = delete(I,suppress)
            count = count + 1
  return pick

## Run detection on example

In [13]:
# load images
dir = '../data/test'
images = glob.glob(os.path.join(dir,'*.jpeg'))
#image = read_image_bgr('../data/test/20290260_Photo 0.jpeg')

for img in images:
    image = read_image_bgr(img)

    # copy to draw on
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    boxes /= scale
    boxes=boxes[0]
    scores = scores[0]
    labels = labels[0]
    keep = nms(boxes,scores, 0.3)
    final_boxes = [boxes[idx] for idx in keep]
    final_scores = [scores[idx] for idx in keep]
    final_labels = [labels[idx] for idx in keep]
    print img
    print("processing time: ", time.time() - start)

    # visualize detections
    for box, score, label in zip(final_boxes, final_scores, final_labels):
        # scores are sorted so we can break
        if score < 0.65:
            break
        #color = label_color(label)
        color = (0,0,255)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    
    #plt.figure(figsize=(15, 15))
    #plt.axis('off')
    #plt.imshow(draw)
    #plt.show()
    cv2.imwrite(os.path.join(dir, '../test_old',os.path.basename(img)), draw)


../data/test/20289972_Photo 0.jpeg
('processing time: ', 1.489799976348877)
../data/test/20296323_Photo 1.jpeg
('processing time: ', 0.3635859489440918)
../data/test/20290055_Photo 0.jpeg
('processing time: ', 0.37131214141845703)
../data/test/20289928_Photo 0.jpeg
('processing time: ', 0.368757963180542)
../data/test/20290062_Photo 0.jpeg
('processing time: ', 0.36928606033325195)
../data/test/20290260_Photo 0.jpeg
('processing time: ', 0.3548460006713867)
../data/test/20289929_Photo 0.jpeg
('processing time: ', 0.33522701263427734)
